This notebook allows you to manually and interactively to estimate FFH from a given GSV pano image, which is useful for ground-truth data assessment and updates (if required) and more reliable than visual assessment. It requires:
* building/eave height measured from building & canopy height raster
* building eave width measured from DSM or building & canopy height raster
* pano id of corresponding building (to allow searching for pano image)
* interactively measured building/eave height in pixels
* interactively measured eave width in pixels
* interactively measured floor height in pixels  

Update the first two parameters each time and re-run all cells.

__Note: There seems bug on using ESC key to close window, so you will need to click on close button to finish each window.__

### Load dependencies

In [19]:
import cv2
import glob
import os

### Set input parameters:
* input pano images folder
* pano image id
* building/eave height (m) - derived from building &canopy height raster
* building eave width (m) - derived from DSM or building &canopy height raster

In [20]:
pano_folder=r"C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\Data Exploration\GSV\Wagga\Pano_clipped_all"

In [21]:
# pano_id='Sdp4ax7QS6z-NBY_iBH5ww' # please update every time you run this notebook
# building_height_m=3.5 # remember to update correspondingly the same feature you are measuring

pano_id='qRwIoP-OssOI6-Czcvn9Xg'
building_height_m=3.5
eave_width_m=1.2 # just an example not measured value

In [22]:
# Load image
pano_file=glob.glob(os.path.join(pano_folder,pano_id+'.jpg'))[0]
original_img = cv2.imread(pano_file)

### Store measured points

In [23]:
# List to store two points
points_buildingheight = []
points_eavewidth = []
points_floorheight = []

dist_buildingheight=[]
dist_eavewidth=[]
dist_floorheight=[]


### defines callback function

In [24]:
def measure_distance(event, x, y, flags, param):
    """Handles mouse clicks to measure X and Y distances."""
    img_ref, points, dists, window_name = param  # Unpack parameters
    # global points, img

    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))

        if len(points) == 2:
            # Calculate distances
            x_dist = abs(points[1][0] - points[0][0])
            y_dist = abs(points[1][1] - points[0][1])

            print(f"Start point coordinates: {points[0]}")
            print(f"End point coordinates: {points[1]}")
            print(f"X Distance: {x_dist} pixels")
            print(f"Y Distance: {y_dist} pixels")
            
            dists.append((x_dist,y_dist))

            # Draw points and line
            cv2.circle(img_ref, points[0], 5, (0, 0, 255), -1)  # Red point
            cv2.circle(img_ref, points[1], 5, (0, 255, 0), -1)  # Green point
            cv2.line(img_ref, points[0], (points[1][0], points[0][1]), (255, 0, 0), 2)  # Horizontal line
            cv2.line(img_ref, points[1], (points[1][0], points[0][1]), (255, 0, 0), 2)  # Vertical line
            # Refresh the window with updated image
            cv2.imshow(window_name, img_ref)

            points = []  # Reset points
            dists = []

### Interactively measure building/eave and floor heights
* three image windows will be open, one for building/eave height, one for eave width and the other for floor height measuring
* single click on two points on each image to measure
* close window to finish

In [25]:
# Initialize active window tracker
active_window = None

def set_active_window(window_name):
    """Updates the active window when mouse interacts with a window."""
    global active_window
    active_window = window_name

def on_mouse(event, x, y, flags, param):
    """Mouse callback that updates the active window."""
    img, points, dist, window_name = param
    if event == cv2.EVENT_LBUTTONDOWN:
        set_active_window(window_name)
    # Your existing measure_distance logic here
    measure_distance(event, x, y, flags, (img, points, dist, window_name))

In [26]:
# Create two windows
window_buildingheight='Measure_building_height_'+pano_id
window_eavewidth='Measure_eave_width_'+pano_id
window_floorheight='Measure_floor_height_'+pano_id
cv2.namedWindow(window_buildingheight)
cv2.namedWindow(window_eavewidth)
cv2.namedWindow(window_floorheight)

# Create copies of the original image for drawing
img_building = original_img.copy()
img_eave = original_img.copy()
img_floor = original_img.copy()

# set mouse callback
cv2.setMouseCallback(window_buildingheight, measure_distance, param=(img_building, points_buildingheight, dist_buildingheight,window_buildingheight))
cv2.setMouseCallback(window_eavewidth, measure_distance, param=(img_eave, points_eavewidth, dist_eavewidth,window_eavewidth))
cv2.setMouseCallback(window_floorheight, measure_distance, param=(img_floor, points_floorheight, dist_floorheight,window_floorheight))


In [27]:
# # Track window status
# building_open = True
# eave_open = True
# floor_open = True

# # Show images and handle closing
# while building_open or eave_open or floor_open:
#     # Only display open windows
#     if building_open and cv2.getWindowProperty(window_buildingheight, cv2.WND_PROP_VISIBLE) >= 1:
#         cv2.imshow(window_buildingheight, img_building)
#     else:
#         building_open = False  # Window is closed manually

#     if eave_open and cv2.getWindowProperty(window_eavewidth, cv2.WND_PROP_VISIBLE) >= 1:
#         cv2.imshow(window_eavewidth, img_eave)
#     else:
#         eave_open = False  # Window is closed manually

#     if floor_open and cv2.getWindowProperty(window_floorheight, cv2.WND_PROP_VISIBLE) >= 1:
#         cv2.imshow(window_floorheight, img_floor)
#     else:
#         floor_open = False  # Window is closed manually

#     key = cv2.waitKey(1) & 0xFF  # Check for key press

#     # Close windows independently when ESC is pressed (order: building → eave → floor)
#     if key == 27:
#         if building_open:  # Close building window first if open
#             cv2.destroyWindow(window_buildingheight)
#             building_open = False
#         elif eave_open:  # Then close eave window if open
#             cv2.destroyWindow(window_eavewidth)
#             eave_open = False
#         elif floor_open:  # Finally close floor window if open
#             cv2.destroyWindow(window_floorheight)
#             floor_open = False

# # Ensure all windows are closed
# cv2.destroyAllWindows()


In [28]:

# Window status tracking
windows = {
    window_buildingheight: True,
    window_eavewidth: True,
    window_floorheight: True
}

while any(windows.values()):
    # Update window status
    for window in list(windows.keys()):
        windows[window] = cv2.getWindowProperty(window, cv2.WND_PROP_VISIBLE) > 0
    
    # Display open windows
    if windows[window_buildingheight]:
        cv2.imshow(window_buildingheight, img_building)
    if windows[window_eavewidth]:
        cv2.imshow(window_eavewidth, img_eave)
    if windows[window_floorheight]:
        cv2.imshow(window_floorheight, img_floor)

    key = cv2.waitKey(1) & 0xFF

    # ESC closes only the active window
    if key == 27 and active_window and windows.get(active_window, False):
        cv2.destroyWindow(active_window)
        windows[active_window] = False
        active_window = None  # Reset after closing

cv2.destroyAllWindows()

Start point coordinates: (1078, 2030)
End point coordinates: (1195, 2030)
X Distance: 117 pixels
Y Distance: 0 pixels
Start point coordinates: (1489, 2254)
End point coordinates: (1492, 2306)
X Distance: 3 pixels
Y Distance: 52 pixels
Start point coordinates: (1645, 1978)
End point coordinates: (1627, 2297)
X Distance: 18 pixels
Y Distance: 319 pixels


### Estimate floor height
* EEH estimation result will be printed out in the end

In [29]:
floor_height_m_1=dist_floorheight[0][1]/dist_buildingheight[0][1]*building_height_m
floor_height_m_2=dist_floorheight[0][1]/dist_eavewidth[0][0]*eave_width_m
print('Estimated floor height (m) using building/eave height:',floor_height_m_1)
print('Estimated floor height (m) using building eave width:',floor_height_m_2)

Estimated floor height (m) using building/eave height: 0.5705329153605015
Estimated floor height (m) using building eave width: 0.5333333333333333
